Importando as bibliotecas e selecionando as colunas no dataframe com o import

In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format= "{:,.2f}".format

In [3]:
cols=['SG_UF_RESIDENCIA','SG_UF_NASCIMENTO','NU_IDADE','TP_SEXO','TP_COR_RACA','TP_ESCOLA','CO_ESCOLA','TP_PRESENCA_CN','TP_PRESENCA_CH',
     'TP_PRESENCA_LC','TP_PRESENCA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','TP_LINGUA','NU_NOTA_REDACAO','Q006','Q025']
df = pd.read_csv("./MICRODADOS_ENEM_2019.csv", encoding='latin-1', sep=";", usecols=cols)

TRATANDO OS DADOS:

Verificando os dados vazios:

In [4]:
df.isna().sum()

SG_UF_RESIDENCIA          0
NU_IDADE                 69
TP_SEXO                   0
TP_COR_RACA               0
SG_UF_NASCIMENTO     145249
TP_ESCOLA                 0
CO_ESCOLA           3947858
TP_PRESENCA_CN            0
TP_PRESENCA_CH            0
TP_PRESENCA_LC            0
TP_PRESENCA_MT            0
NU_NOTA_CN          1384837
NU_NOTA_CH          1172125
NU_NOTA_LC          1172125
NU_NOTA_MT          1384837
TP_LINGUA                 0
NU_NOTA_REDACAO     1172126
Q006                      0
Q025                      0
dtype: int64

Neste caso, os dados faltantes na idade podem ser tratados, pela baixa quantidade em relação à base (será utilizada a mediana, pela interferência dos outliers na média). Os vazios em notas são as abstenções e serão mantidos para a análise. Os vazios no campo de escolas são escolas sem cadastro ou com informações erradas, e devido à grande quantidade a coluna vai ser dropada.

In [5]:
df.drop(['CO_ESCOLA'], axis=1, inplace=True)
df['NU_IDADE']=df.NU_IDADE.fillna(df.NU_IDADE.median())

Serão verificados os candidatos eliminados nas provas. Estes serão dropados.

In [6]:
df.drop(df[df.TP_PRESENCA_CN == 2].index, inplace=True)
df.drop(df[df.TP_PRESENCA_CH == 2].index, inplace=True)

Será criada uma coluna com a nota total dos alunos, e após isso as notas individuais por prova e as colunas de presença serão dropadas

In [7]:
df['Total']=(df['NU_NOTA_CN']+df['NU_NOTA_CH']+df['NU_NOTA_LC']+df['NU_NOTA_MT']+df['NU_NOTA_REDACAO'])/5
df.drop(['TP_PRESENCA_CN','TP_PRESENCA_CH','TP_PRESENCA_LC','TP_PRESENCA_MT','NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC',
         'NU_NOTA_MT','NU_NOTA_REDACAO'], axis=1, inplace=True)

Serão verificados os impactos dos dados faltantes no estado de nascimento:

In [8]:
df[df.SG_UF_NASCIMENTO.isna()]['Total'].describe()

count   106,197.00
mean        492.01
std          72.85
min           0.00
25%         444.72
50%         485.50
75%         533.18
max         806.24
Name: Total, dtype: float64

In [9]:
df['Total'].describe()

count   3,702,007.00
mean          522.62
std            83.65
min             0.00
25%           464.04
50%           515.02
75%           576.76
max           850.82
Name: Total, dtype: float64

Percebe-se que a nota máxima no grupo selecionado não está entre as máximas do dataframe total, então não vai modificar as análises caso sejam dropados os nulos da coluna de estado de nascimento

In [10]:
df.dropna(subset=['SG_UF_NASCIMENTO'], inplace=True)

Para o tratamento dos dados faltantes no campo de cor/raça, serão considerados os estados de nascimento para a realização da estimativa

Primeiramente é criada uma cópia dos dados originais, e então as linhas com dados não declarados são dropadas. Os dados restantes são agrupados por estados e uma função lambda é aplicada para apontar a porcentagem na qual os dados restantes estão distribuídos de acordo com os estados

In [12]:
df2=df.copy()
df2.drop(df2[df2.TP_COR_RACA == 0].index, inplace=True)
porcentagens=df2.groupby(['SG_UF_NASCIMENTO','TP_COR_RACA']).agg({'TP_COR_RACA':'count'}).groupby(level=0).apply(lambda x:100*x/float(x.sum()))

In [13]:
porcentagens

TP_COR_RACA
SG_UF_NASCIMENTO TP_COR_RACA             
AC               1                  15.33
                 2                  11.27
                 3                  69.34
                 4                   3.26
                 5                   0.80
...                                   ...
TO               1                  18.52
                 2                  16.52
                 3                  60.72
                 4                   3.58
                 5                   0.67

[135 rows x 1 columns]

É criada então outra cópia dos dados onde serão mantidos apenas os dados não informados, para quantificarmos a quantidade a ser preenchida para cada estado

In [14]:
df3=df.copy()
df3.drop(df3[df3.TP_COR_RACA != 0].index, inplace=True)
nulos=df3.groupby(['SG_UF_NASCIMENTO'])[['TP_COR_RACA']].count()

In [15]:
nulos

,TP_COR_RACA
SG_UF_NASCIMENTO,
AC,705
AL,2096
AM,2414
AP,660
BA,9563
CE,8059
DF,2430
ES,1453
GO,4227


Por fim, estes dois datasets criados são mesclados, criando um dataset que auxiliará no preenchimento do dataframe original, demonstrando a quantidade de linhas a ser populada com cada valor em cada estado

In [16]:
merged=porcentagens.merge(nulos, how="outer", on="SG_UF_NASCIMENTO")
merged['DISTRIBUICAO']=round(merged['TP_COR_RACA_y']*(merged['TP_COR_RACA_x']/100)).astype('int32')
merged['COR_RACA']=((merged.groupby('SG_UF_NASCIMENTO').cumcount()%5)+1)
merged['DONE']=0
merged.drop(['TP_COR_RACA_x','TP_COR_RACA_y'], axis=1, inplace=True)

A coluna DONE é criada e populada com zeros, e será utilizada como base para a execução da função de preenchimento do dataframe original

In [17]:
#Antes de Executar
merged

,DISTRIBUICAO,COR_RACA,DONE
SG_UF_NASCIMENTO,,,
AC,108,1,0
AC,79,2,0
AC,489,3,0
AC,23,4,0
AC,6,5,0
...,...,...,...
TO,146,1,0
TO,130,2,0
TO,478,3,0


É criada uma função que vai iterar todo o dataframe buscando os valores não preenchidos de cor/raça, e vai preencher estes dados de acordo com a distribuição por estado presente no dataset auxiliar

In [18]:
df4=df.copy()
for row in df4.itertuples():   
    if row[4]==0:
        count=1
        while True:
            state=row[5]
            if (merged.loc[(merged.index == state) & (merged['COR_RACA'] == count)]['DISTRIBUICAO'] > merged.loc[(merged.index == state) & (merged['COR_RACA'] == count)]['DONE']).all():
                df4.at[row.Index,'TP_COR_RACA'] = count
                merged.loc[(merged.index == state) & (merged['COR_RACA'] == count),'DONE']+=1
                break
            else:
                count+=1
            if count>5:
                df4.at[row.Index,'TP_COR_RACA'] = 3
                break

In [19]:
#Depois de Executar
merged

,DISTRIBUICAO,COR_RACA,DONE
SG_UF_NASCIMENTO,,,
AC,108,1,108
AC,79,2,79
AC,489,3,489
AC,23,4,23
AC,6,5,6
...,...,...,...
TO,146,1,146
TO,130,2,130
TO,478,3,478


É demonstrada então uma comparação entre as quantidades preenchidas por valor da coluna cor/sexo, antes e depois da execução da função

In [20]:
#Original
df['TP_COR_RACA'].value_counts()

3    2286853
1    1790822
2     626954
4     109961
0      99676
5      29952
Name: TP_COR_RACA, dtype: int64

In [21]:
#Populado cor/raça
df4['TP_COR_RACA'].value_counts()

3    2334750
1    1826479
2     640140
4     112273
5      30576
Name: TP_COR_RACA, dtype: int64

Tratados os dados de cor/raça, faremos o mesmo procedimento para tratar os dados de escola, mas com os dados do estado de residência

In [22]:
df5=df.copy()
df5.drop(df5[df5.TP_ESCOLA == 1].index, inplace=True)
porcentagens2=df5.groupby(['SG_UF_RESIDENCIA','TP_ESCOLA']).agg({'TP_ESCOLA':'count'}).groupby(level=0).apply(lambda x:100*x/float(x.sum()))

In [23]:
porcentagens2

TP_ESCOLA
SG_UF_RESIDENCIA TP_ESCOLA           
AC               2              94.42
                 3               5.58
AL               2              81.16
                 3              18.84
AM               2              94.50
                 3               5.50
AP               2              92.10
                 3               7.90
BA               2              85.37
                 3              14.63
CE               2              92.21
                 3               7.79
DF               2              75.34
                 3              24.66
ES               2              88.87
                 3              11.13
GO               2              85.25
                 3              14.75
MA               2              93.10
                 3               6.90
MG               2              87.95
                 3              12.05
MS               2              90.12
                 3               9.88
MT               2              90.43
                 3               9.57
PA               2              87.79
                 3              12.21
PB               2              83.29
                 3              16.71
PE               2              86.07
                 3              13.93
PI               2              86.49
                 3              13.51
PR               2              84.82
                 3              15.18
RJ               2              73.49
                 3              26.51
RN               2              83.95
                 3              16.05
RO               2              95.99
                 3               4.01
RR               2              91.72
                 3               8.28
RS               2              86.46
                 3              13.54
SC               2              86.30
                 3              13.70
SE               2              78.98
                 3              21.02
SP               2              79.18
                 3              20.82
TO               2              93.19
                 3               6.81

In [25]:
df6=df.copy()
df6.drop(df6[df6.TP_ESCOLA != 1].index, inplace=True)
nulos2=df6.groupby(['SG_UF_RESIDENCIA'])[['TP_ESCOLA']].count()

In [26]:
nulos2

,TP_ESCOLA
SG_UF_RESIDENCIA,
AC,30436
AL,67129
AM,84453
AP,33109
BA,295670
CE,173008
DF,65932
ES,70827
GO,117126


In [27]:
merged2=porcentagens2.merge(nulos2, how="outer", on="SG_UF_RESIDENCIA")
merged2['DISTRIBUICAO']=round(merged2['TP_ESCOLA_y']*(merged2['TP_ESCOLA_x']/100)).astype('int64')
merged2['ESCOLA']=((merged2.groupby('SG_UF_RESIDENCIA').cumcount()%2)+2)
merged2['DONE']=0
merged2.drop(['TP_ESCOLA_x','TP_ESCOLA_y'], axis=1, inplace=True)

In [28]:
#antes de executar
merged2

,DISTRIBUICAO,ESCOLA,DONE
SG_UF_RESIDENCIA,,,
AC,28738,2,0
AC,1698,3,0
AL,54485,2,0
AL,12644,3,0
AM,79808,2,0
AM,4645,3,0
AP,30494,2,0
AP,2615,3,0
BA,252422,2,0


In [29]:
df7=df4.copy()
for row in df7.itertuples():   
    if row[6]==1:
        count=2
        while True:
            state=row[1]
            if (merged2.loc[(merged2.index == state) & (merged2['ESCOLA'] == count)]['DISTRIBUICAO'] > merged2.loc[(merged2.index == state) & (merged2['ESCOLA'] == count)]['DONE']).all():
                df7.at[row.Index,'TP_ESCOLA'] = count
                merged2.loc[(merged2.index == state) & (merged2['ESCOLA'] == count),'DONE']+=1
                break
            else:
                count+=1
            if count>3:
                df7.at[row.Index,'TP_ESCOLA'] = 2
                break

In [31]:
#depois de executar
merged2

,DISTRIBUICAO,ESCOLA,DONE
SG_UF_RESIDENCIA,,,
AC,28738,2,28738
AC,1698,3,1698
AL,54485,2,54485
AL,12644,3,12644
AM,79808,2,79808
AM,4645,3,4645
AP,30494,2,30494
AP,2615,3,2615
BA,252422,2,252422


In [32]:
#Original
df['TP_ESCOLA'].value_counts()

1    3528141
2    1202118
3     213959
Name: TP_ESCOLA, dtype: int64

In [35]:
#Após popular escola
df7['TP_ESCOLA'].value_counts()

2    4209517
3     734701
Name: TP_ESCOLA, dtype: int64

por fim, salvamos o novo dataset em csv para evitar ter que realizar estes procedimentos novamente

In [43]:
df7.to_csv('dados_populados.csv', index=False)

DADOS TRATADOS

ANÁLISES:

uma vez tratados os dados, podemos iniciar as análises importando o dataset salvo:

In [20]:
enem = pd.read_csv("./dados_populados.csv", encoding='latin-1')

realizamos então a mudança nas variáveis, para facilitar a leitura

In [21]:
enem['TP_COR_RACA'] = enem['TP_COR_RACA'].replace([1,2,3,4,5],['Branco','Preto','Pardo','Amarelo','Indígena'])
enem['TP_ESCOLA'] = enem['TP_ESCOLA'].replace([2,3],['Publica','Privada'])
enem['TP_LINGUA']=enem['TP_LINGUA'].replace([0,1],['Ingles','Espanhol'])
enem['Q006']=enem['Q006'].replace(['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q'],[
    'A (0)','B (0 a 1)','C (1 a 1,5)','D (1,5 a 2)','E (2 a 2,5)','F (2,5 a 3)','G (3 a 4)','H (4 a 5)','I (5 a 6)','J (6 a 7)','K (7 a 8)','L (8 a 9)',
    'M (9 a 10)','N (10 a 12)','O (12 a 15)','P (15 a 20)','Q (Mais de 20)'])
enem['Q025']=enem['Q025'].replace(['A','B'],['Nao','Sim'])
enem['NU_IDADE']=enem['NU_IDADE'].astype('int32')
enem.rename(columns={'SG_UF_RESIDENCIA': 'UF_RESIDENCIA', 'NU_IDADE': 'IDADE', 'TP_SEXO':'SEXO','TP_COR_RACA':'COR/RACA', 'SG_UF_NASCIMENTO':'UF_NASCIMENTO','TP_ESCOLA':'ESCOLA_ENS_MEDIO','TP_LINGUA':'LINGUA_ESTR','Q006':'RENDA_FAMILIAR(SALARIOS)','Q025':'ACESSO_INTERNET'}, inplace=True)


In [22]:
enem

,UF_RESIDENCIA,IDADE,SEXO,COR/RACA,UF_NASCIMENTO,ESCOLA_ENS_MEDIO,LINGUA_ESTR,RENDA_FAMILIAR(SALARIOS),ACESSO_INTERNET,Total
0,SP,36,M,Pardo,BA,Publica,Ingles,A (0),Sim,NaN
1,BA,23,M,Branco,BA,Publica,Ingles,"C (1 a 1,5)",Sim,NaN
2,CE,39,F,Pardo,CE,Publica,Espanhol,B (0 a 1),Sim,NaN
3,TO,25,F,Branco,TO,Publica,Espanhol,"E (2 a 2,5)",Sim,475.34
4,MG,22,F,Branco,MG,Publica,Ingles,G (3 a 4),Sim,639.36
...,...,...,...,...,...,...,...,...,...,...
4944213,DF,41,F,Pardo,PA,Privada,Ingles,"C (1 a 1,5)",Sim,NaN
4944214,AM,20,M,Pardo,AM,Privada,Espanhol,"C (1 a 1,5)",Sim,NaN
4944215,RS,21,M,Indígena,RS,Privada,Ingles,"C (1 a 1,5)",Sim,NaN
4944216,RS,22,M,Indígena,RS,Privada,Ingles,A (0),Sim,NaN


O Foco desta análise será relacionar as características de etnia e condição socioeconômica dos participantes com seus respectivos desempenhos na prova.

Iniciamos então buscando saber a porcentagem de candidatos de cada faixa de renda inscritos no processo

In [29]:
enem.groupby(['RENDA_FAMILIAR(SALARIOS)']).agg({'RENDA_FAMILIAR(SALARIOS)':'count'}).apply(lambda x:100*x/float(x.sum()))

,RENDA_FAMILIAR(SALARIOS)
RENDA_FAMILIAR(SALARIOS),
A (0),4.49
B (0 a 1),24.77
"C (1 a 1,5)",25.67
"D (1,5 a 2)",9.84
"E (2 a 2,5)",9.37
"F (2,5 a 3)",4.47
G (3 a 4),5.97
H (4 a 5),3.83
I (5 a 6),2.85


Neste ponto ja retiramos uma informação importante: quase 55% dos cerca de 5 milhões de inscritos no processo possuem uma renda familiar de até 1,5 salários mínimos. Aumentando a faixa para 2,5 salários, temos quase 75% dos inscritos...

Podemos também fazer esse cálculo para as regiões do Brasil. Para isso criamos uma coluna auxiliar com a região de cada inscrito, e aplicamos o mesmo procedimento

In [30]:
def regiao(x):
    if x == 'PR' or x == 'SC' or x == 'RS':
        return "Sul"
    elif x == 'SP' or x== 'RJ' or x == 'MG' or x == 'ES':
        return "Sudeste"
    elif x == 'GO' or x== 'MT' or x == 'MS' or x== 'DF':
        return "Centro-Oeste"
    elif x == 'AL' or x== 'BA' or x == 'CE' or x== 'MA' or x=='PB' or x=='PE' or x=='PI' or x=='RN':
        return "Nordeste"
    elif x == 'AC' or x=='AP' or x=='AM' or x=='PA' or x=='RO' or x=='RR' or x=='TO':
        return "Norte"

In [33]:
enem['REGIAO']=enem['UF_RESIDENCIA'].apply(regiao)
renda=enem.groupby(['REGIAO','RENDA_FAMILIAR(SALARIOS)']).agg({'RENDA_FAMILIAR(SALARIOS)':'count'}).groupby(level=0).apply(lambda x:100*x/float(x.sum()))

Vamos então comparar os inscritos da região sudeste com a região nordeste:

In [70]:
renda.loc[['Nordeste']]

RENDA_FAMILIAR(SALARIOS)
REGIAO   RENDA_FAMILIAR(SALARIOS)                          
Nordeste A (0)                                         7.53
         B (0 a 1)                                    39.11
         C (1 a 1,5)                                  25.99
         D (1,5 a 2)                                   7.08
         E (2 a 2,5)                                   6.07
         F (2,5 a 3)                                   2.55
         G (3 a 4)                                     3.28
         H (4 a 5)                                     2.12
         I (5 a 6)                                     1.55
         J (6 a 7)                                     0.88
         K (7 a 8)                                     0.62
         L (8 a 9)                                     0.47
         M (9 a 10)                                    0.46
         N (10 a 12)                                   0.67
         O (12 a 15)                                   0.55
         P (15 a 20)                                   0.47
         Q (Mais de 20)                                0.61

In [53]:
renda.loc[['Sudeste']]

RENDA_FAMILIAR(SALARIOS)
REGIAO  RENDA_FAMILIAR(SALARIOS)                          
Sudeste A (0)                                         2.32
        B (0 a 1)                                    13.85
        C (1 a 1,5)                                  24.75
        D (1,5 a 2)                                  11.77
        E (2 a 2,5)                                  11.98
        F (2,5 a 3)                                   5.93
        G (3 a 4)                                     8.17
        H (4 a 5)                                     5.19
        I (5 a 6)                                     3.88
        J (6 a 7)                                     2.17
        K (7 a 8)                                     1.58
        L (8 a 9)                                     1.21
        M (9 a 10)                                    1.20
        N (10 a 12)                                   1.72
        O (12 a 15)                                   1.39
        P (15 a 20)                                   1.22
        Q (Mais de 20)                                1.66

Aqui temos uma diferença perceptível: enquanto na região nordeste mais de 70% dos inscritos possuem renda familiar menor que 1,5 salários, na região sudeste esta porcentagem cai para 40%

E essa diferença impacta a pontuação total de alguma forma?

In [54]:
enem.groupby(['RENDA_FAMILIAR(SALARIOS)'])['Total'].mean()

RENDA_FAMILIAR(SALARIOS)
A (0)            473.64
B (0 a 1)        483.27
C (1 a 1,5)      504.22
D (1,5 a 2)      523.08
E (2 a 2,5)      531.06
F (2,5 a 3)      547.23
G (3 a 4)        555.57
H (4 a 5)        571.81
I (5 a 6)        580.24
J (6 a 7)        590.71
K (7 a 8)        595.84
L (8 a 9)        603.26
M (9 a 10)       605.87
N (10 a 12)      613.71
O (12 a 15)      622.02
P (15 a 20)      629.54
Q (Mais de 20)   637.72
Name: Total, dtype: float64

Percebe-se que a média da pontuação total dos candidatos nas provas cresce à medida em que cresce a renda familiar. Isso pode se dar pela opção por melhores escolas, bem como mais tempo para estudar com apoio financeiro da família

E como se comportam as notas de acordo com a região, uma vez que há a disparidade de renda?

In [56]:
enem.groupby(['REGIAO'])['Total'].mean().sort_values()

REGIAO
Norte          496.08
Nordeste       509.43
Centro-Oeste   522.65
Sul            536.84
Sudeste        542.49
Name: Total, dtype: float64

Como esperado, percebe-se que a média das pontuações dos candidatos aumenta nas regiões onde se concentra a maior renda. Podemos fazer a mesma análise para cada estado:

In [58]:
enem.groupby(['UF_RESIDENCIA'])['Total'].mean().sort_values(ascending=False)

UF_RESIDENCIA
SP   544.41
MG   541.49
RJ   541.32
SC   540.70
DF   539.02
RS   536.39
ES   536.32
PR   535.29
GO   524.56
RN   521.72
SE   517.25
PE   516.16
CE   514.88
MS   514.87
PB   514.56
BA   508.10
MT   506.79
PI   504.11
AL   503.66
RR   502.79
TO   500.20
PA   498.92
RO   497.84
AC   493.15
MA   491.04
AP   489.83
AM   488.05
Name: Total, dtype: float64

É perceptível que entre os 10 estados com a maior média, 7 são os estados das regiões sudeste e sul, além de 2 da região centro-oeste, que são os estados que apresentam as maiores rendas para os alunos

Olhando por outro ângulo, qual a distribuição das diversas etnias declaradas de acordo com cada região?

In [66]:
enem.groupby(['REGIAO','COR/RACA']).agg({'COR/RACA':'count'}).groupby(level=0).apply(lambda x:100*x/float(x.sum()))

COR/RACA
REGIAO       COR/RACA          
Centro-Oeste Amarelo       3.19
             Branco       34.43
             Indígena      0.79
             Pardo        49.47
             Preto        12.12
Nordeste     Amarelo       2.58
             Branco       22.48
             Indígena      0.75
             Pardo        58.50
             Preto        15.69
Norte        Amarelo       2.22
             Branco       16.52
             Indígena      1.20
             Pardo        68.94
             Preto        11.12
Sudeste      Amarelo       2.06
             Branco       47.52
             Indígena      0.39
             Pardo        36.80
             Preto        13.22
Sul          Amarelo       1.21
             Branco       73.66
             Indígena      0.21
             Pardo        19.19
             Preto         5.73

Percebe-se uma grande diferença entre algumas regiões do país, com o Sul registrando mais de 70% declarados brancos com apenas 25% pretos e pardos, enquanto no norte os dados se invertem, com 80% de pretos e pardos e pouco mais de 15% de brancos

E há alguma influência da cor/raça declarada na média final dos candidatos?

In [76]:
enem.groupby(['COR/RACA'])['Total'].mean().sort_values(ascending=False)

COR/RACA
Branco     549.30
Amarelo    520.89
Pardo      508.40
Preto      503.73
Indígena   476.91
Name: Total, dtype: float64

Conforme imaginado, há uma grande diferença nos desempenhos de candidatos declarados brancos em relação a candidatos declarados pretos ou pardos. Uma hipótese para esta discrepância é a ja conhecida desigualdade brasileira, com pessoas declaradas pretas ou pardas ocupando posições de mais baixa renda, enquanto pessoas declaradas brancas contam com mais privilégios. A situação é ainda mais difícil quando analisamos os indígenas, que muitas vezes passam por um processo de difícil integração à sociedade

Apenas para exemplificação, vejamos as diferenças na renda declarada dos candidatos declarados brancos para os declarados pretos/pardos:

In [73]:
renda_cor = enem.groupby(['COR/RACA','RENDA_FAMILIAR(SALARIOS)']).agg({'RENDA_FAMILIAR(SALARIOS)':'count'}).groupby(level=0).apply(lambda x:100*x/float(x.sum()))
renda_cor.loc[['Branco']]

RENDA_FAMILIAR(SALARIOS)
COR/RACA RENDA_FAMILIAR(SALARIOS)                          
Branco   A (0)                                         2.54
         B (0 a 1)                                    14.47
         C (1 a 1,5)                                  21.09
         D (1,5 a 2)                                  10.54
         E (2 a 2,5)                                  10.73
         F (2,5 a 3)                                   5.92
         G (3 a 4)                                     8.18
         H (4 a 5)                                     5.84
         I (5 a 6)                                     4.53
         J (6 a 7)                                     2.66
         K (7 a 8)                                     1.96
         L (8 a 9)                                     1.55
         M (9 a 10)                                    1.55
         N (10 a 12)                                   2.30
         O (12 a 15)                                   1.93
         P (15 a 20)                                   1.75
         Q (Mais de 20)                                2.46

In [74]:
renda_cor.loc[['Preto','Pardo']]

RENDA_FAMILIAR(SALARIOS)
COR/RACA RENDA_FAMILIAR(SALARIOS)                          
Preto    A (0)                                         5.55
         B (0 a 1)                                    30.97
         C (1 a 1,5)                                  30.18
         D (1,5 a 2)                                   9.65
         E (2 a 2,5)                                   8.75
         F (2,5 a 3)                                   3.40
         G (3 a 4)                                     4.41
         H (4 a 5)                                     2.31
         I (5 a 6)                                     1.59
         J (6 a 7)                                     0.78
         K (7 a 8)                                     0.54
         L (8 a 9)                                     0.37
         M (9 a 10)                                    0.35
         N (10 a 12)                                   0.41
         O (12 a 15)                                   0.32
         P (15 a 20)                                   0.21
         Q (Mais de 20)                                0.19
Pardo    A (0)                                         5.63
         B (0 a 1)                                    30.84
         C (1 a 1,5)                                  27.93
         D (1,5 a 2)                                   9.38
         E (2 a 2,5)                                   8.56
         F (2,5 a 3)                                   3.68
         G (3 a 4)                                     4.76
         H (4 a 5)                                     2.72
         I (5 a 6)                                     1.94
         J (6 a 7)                                     1.03
         K (7 a 8)                                     0.71
         L (8 a 9)                                     0.51
         M (9 a 10)                                    0.48
         N (10 a 12)                                   0.62
         O (12 a 15)                                   0.48
         P (15 a 20)                                   0.36
         Q (Mais de 20)                                0.38

É perceptível o distanciamento das realidades sociais, onde menos de 40% dos candidatos possuem renda familiar de menos de 1,5 salários, e mais de 2% dos inscritos possuem renda superior a 20 salários. No caso de pretos e pardos, os valores chegam a 65% vivendo com até 1,5 salários, e ambas somadas não chegam a 0,6% de candidatos com mais de 20 salários. Isso comprova a desigualdade de renda baseada na cor, realidade vivenciada atualmente.

Por fim, vejamos como se comportam os candidatos que possuem as 10 maiores notas:

In [77]:
enem.sort_values(by='Total', ascending=False).head(10)

,UF_RESIDENCIA,IDADE,SEXO,COR/RACA,UF_NASCIMENTO,ESCOLA_ENS_MEDIO,LINGUA_ESTR,RENDA_FAMILIAR(SALARIOS),ACESSO_INTERNET,Total,REGIAO
2527191,MG,17,M,Branco,MG,Privada,Ingles,N (10 a 12),Sim,850.82,Sudeste
2328495,MG,17,M,Branco,MG,Publica,Ingles,N (10 a 12),Sim,846.26,Sudeste
478276,PI,18,M,Branco,PI,Privada,Ingles,J (6 a 7),Sim,845.00,Nordeste
1753470,MA,18,M,Pardo,PI,Publica,Ingles,J (6 a 7),Sim,843.28,Nordeste
2112831,GO,17,F,Branco,GO,Privada,Ingles,P (15 a 20),Sim,842.70,Centro-Oeste
2384881,GO,18,M,Pardo,GO,Privada,Ingles,H (4 a 5),Sim,840.56,Centro-Oeste
2951796,GO,17,F,Branco,GO,Privada,Ingles,N (10 a 12),Sim,838.56,Centro-Oeste
2177101,MG,19,M,Branco,MG,Publica,Ingles,Q (Mais de 20),Sim,837.48,Sudeste
2603783,SP,22,F,Branco,SP,Publica,Ingles,J (6 a 7),Sim,835.90,Sudeste
3041128,GO,18,M,Branco,GO,Publica,Ingles,K (7 a 8),Sim,833.58,Centro-Oeste


Seguindo as hipóteses apresentadas anteriormente, percebe-se que entre as 10 maiores notas, 80% são de candidatos declarados brancos. Apenas 20% são da região nordeste, e nenhum deles possui renda familiar inferior a 4 salários, com a grande maioria acima de 6 salários...

Podemos perceber assim o impacto que a desigualdade de renda gera no desempenho dos estudantes, desigualdade essa que vem aliada à desigualdade de cor/raça e às distribuições geográficas no Brasil. A hipótese seguida para justificar tal resultado é a dificuldade no incentivo a uma educação de qualidade vindo das parcelas da sociedade de baixa renda, com muitos necessitando mesclar estudos e trabalho ou enfrentar condições ruins de educação. Por sua vez, estudantes de famílias de mais alta renda geralmente possuem uma liberdade maior para focar nos estudos, além do acesso a uma educação de maior qualidade. Realidade essa que deve ser revista, uma vez que a dificuldade do acesso à educação apenas perpetuará essa situação de desigualdade.